In [ ]:
pip install faker sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from datetime import datetime
from sqlalchemy.orm import Session, sessionmaker, scoped_session
from faker import Faker
from uuid import uuid4
import random

In [ ]:
!pwd

/content


In [ ]:
fake = Faker('pt_BR')  # criando gerador de dados falsos

In [ ]:
engine = create_engine("sqlite:///foo.db", echo=False)  # conectando ao banco de dados
Base = declarative_base()  # declarando as estruturas, cria as tabelas dentro da classe

class Pessoas(Base):
    __tablename__ = "pessoas"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    email = Column(String)
    birth = Column(DateTime)
    job = Column(String)

class Pessoas2(Base):
    __tablename__ = "pessoas2"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String,index=True)  # index para deixar as consultas mais rapidas
    email = Column(String, index=True, unique=True)  # unique para ter um unico e-mail
    birth = Column(DateTime)
    job = Column(String)

In [ ]:
Pessoas.__table__.drop(engine, checkfirst = True)

In [ ]:
Pessoas2.__table__.drop(engine, checkfirst = True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)  # criando uma sessao para manipular os dados(exemplo: select, insert e update)
session = Session()

## Comparando tempo de execução da tabela pessoas1(sem o uso do index) da tabela pessoas2(com o uso do index)###  


**1. Inserindo 9999 registros na tabela Pessoas:**

In [ ]:
%%time
# inserindo registros no banco
# cronometrando usando o comando time     
for i in range(9999):  # inserindo 9999 registros
  insert_fake = Pessoas(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 17.3 s, sys: 4.91 s, total: 22.2 s
Wall time: 1min 43s


In [ ]:
result = session.query(Pessoas).count()  # count para confirmar a quantidade de registros
result

9999

In [ ]:
# instanciando registros
julio_user = Pessoas(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")  # fromisoformat: padrão internacional de data e hora
session.add(julio_user)

In [ ]:
# cronometrando usando o comando timeit, para ver o tempo medio da execucao
%%timeit
session.query(Pessoas).filter_by(name="Julio").all()

1.34 ms ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**2. Inserindo 9999 registros na tabela Pessoas2:**

In [ ]:
%%time
for i in range(9999):
  insert_fake = Pessoas2(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 18.2 s, sys: 5.19 s, total: 23.4 s
Wall time: 1min 48s


In [ ]:
julio_user2 = Pessoas2(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user2)

In [ ]:
%%timeit
session.query(Pessoas2).filter_by(name="Julio").all()

409 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**3. Inserindo 5999 registros na tabela Pessoas:**

In [ ]:
%%time
for i in range(5999):
  insert_fake = Pessoas(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 10.4 s, sys: 2.8 s, total: 13.2 s
Wall time: 1min


In [ ]:
julio_user = Pessoas(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user)

In [ ]:
%%timeit
session.query(Pessoas).filter_by(name="Julio").all()

2.63 ms ± 90.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**4. Inserindo 5999 registros na tabela Pessoas2**

In [ ]:
%%time
for i in range(5999):
  insert_fake = Pessoas2(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 10.7 s, sys: 3.12 s, total: 13.8 s
Wall time: 1min 1s


In [ ]:
julio_user2 = Pessoas2(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user2)

In [ ]:
%%timeit
session.query(Pessoas2).filter_by(name="Julio").all()

440 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## **Conclusão**

### Com a inserção dos 5999 registros, o tempo de inserção da tabela Pessoas(sem o index) foi de 1.34 ms para 2.6 ms. Enquanto o tempo de inserção da tabela Pessoas2(com o uso do index) foi de 404 µs para 420 µs.

### Concluindo que quando temos a coluna indexada e fazemos pesquisa por ela, o tempo é praticamente constante 


**Visualizando registros:**

In [ ]:
for (name,email,birth,job) in session.query(Pessoas.name,Pessoas.email,Pessoas.birth,Pessoas.job).limit(10):
  print(name,email,birth,job)

Sr. Enzo Gabriel Cavalcanti 308e1bd1-3bb6-4feb-81ae-faf7762b782fmaite30@uol.com.br 1968-05-08 00:00:00 Recepcionista
Rodrigo Nascimento dde07ecb-a199-4b11-822c-10a5f39ce309azevedopedro-henrique@bol.com.br 1983-02-05 00:00:00 Dedetizador
Lorena Azevedo 336a34c3-3fae-4f53-a333-fe8470a7231dfernando78@ig.com.br 2014-11-01 00:00:00 Amolador de ferramentas
Valentina Costela d2af9814-0045-42e0-b7dc-51899e1e918amanuela09@gmail.com 2020-05-16 00:00:00 Brigadeiro
Sofia Porto 59f56d18-78bd-40ef-bb4f-03b5cc87bc0dzcorreia@ig.com.br 1993-09-20 00:00:00 Telegrafista
Benício Peixoto 783c2e48-6adc-4b0f-aaf6-8d8ffca21b8fda-matapedro-miguel@ig.com.br 1923-12-28 00:00:00 Meeiro
Erick Azevedo ca159d5c-3853-41d9-bce1-5046bc351511rteixeira@yahoo.com.br 1933-06-18 00:00:00 Leiloeiro
Davi Lucas Moreira c5d0a5c8-b233-461c-9541-24927127e96exaragao@ig.com.br 1918-11-24 00:00:00 Engenheiro mecatrônico
Rafaela Ribeiro 90eaa133-fd9d-4ac8-ba1e-d9656d61e1f6da-costamarina@uol.com.br 1965-08-17 00:00:00 Major
Srta. Bren